In [1]:
from openai import AsyncOpenAI
from dotenv import load_dotenv
import os
import time

In [2]:
SYSTEM_PROMPT = """
상담 스크립트에서 고객의 성향을 분류하세요.
전체적인 맥락을 참고하되 판단의 근거는 고객의 발화에 한정합니다.

### 분류 규칙
1. 제시된 성향 키워드 중 가장 적절한 한가지만 선택한다
2. 오직 키워드만 추출 (예: S8)

### 성향 키워드 목록
- N1 (일반친절형): 전반적으로 상담에 협조적임
- N2 (조용한내성형): 간결한 답변을 하고 원함
- N3 (실용주의형): 불필요한 말 없이 목적 달성에만 집중함
- N4 (친화적수다형): 사적인 이야기나 본인 상황을 길게 설명함
- S1 (급한성격형): 빠른 결론과 처리를 선호함
- S2 (꼼꼼상세형): 상세한 설명을 요구함
- S3 (감정호소형): 본인의 사정을 감정적으로 호소하며 공감을 구함
- S4 (시니어친화형): 정보 기기 사용이 서툴고 설명을 잘 이해하지 못함
- S5 (디지털네이티브): 앱 사용에 능숙하며 전문 용어나 신조어 사용에 거부감이 없음
- S6 (VIP고객형): 본인의 등급이나 기여도를 언급하며 특별한 대우를 기대함
- S7 (반복민원형): 과거의 상담 이력을 언급하며 해결되지 않은 불만을 반복 제기함
- S8 (불만항의형): 강한 어조, 분노, 짜증을 드러내며 서비스나 규정에 대해 항의함
"""

In [3]:
load_dotenv()

client = AsyncOpenAI(
    base_url=os.getenv("RUNPOD_URL"),
    api_key=os.getenv("API_KEY")
)

async def get_personality(script):
    try:
        start_time = time.perf_counter() # 전체 시작 시간
        first_token_time = None
        full_content = ""
        
        # 스트리밍 호출 활성화
        response = await client.chat.completions.create(
            model="local-model",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"상담 전문:\n{script}"}
            ],
            temperature=0.0,
            response_format={"type": "json_object"},
            stream=True,                            # 스트리밍 활성화
            stream_options={"include_usage": True}  # 토큰 사용량 포함
        )

        total_tokens = 0
        async for chunk in response:
            # TTFT 계산 (첫 번째 토큰이 들어온 시점)
            if first_token_time is None and chunk.choices and chunk.choices[0].delta.content:
                first_token_time = time.perf_counter()
            
            # 내용 합치기
            if chunk.choices and chunk.choices[0].delta.content:
                full_content += chunk.choices[0].delta.content
            
            # 토큰 수
            if chunk.usage:
                total_tokens = chunk.usage.completion_tokens

        end_time = time.perf_counter()
        
        # 지표 계산
        ttft = first_token_time - start_time if first_token_time else 0
        total_latency = end_time - start_time
        
        # TPS 계산 (생성된 토큰 수 / 생성에 걸린 시간)
        if total_tokens == 0:
            total_tokens = len(full_content) / 1.5 
            
        tps = total_tokens / total_latency if total_latency > 0 else 0
        
        metrics = {
            "ttft": round(ttft, 3),
            "tps": round(tps, 2),
            "total_tokens": int(total_tokens)
        }

        return full_content, metrics

    except Exception as e:
        return {"error": f"오류 발생: {str(e)}"}, {"ttft": 0, "tps": 0, "total_tokens": 0}

In [6]:
import os
import json
import glob

# 파일 경로 설정
testset_dir = "./testsets"
json_files = glob.glob(os.path.join(testset_dir, "*.json"))

results_list = []

# 파일 루프 시작
for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data_list = json.load(f)
        
    for item in data_list:
        # 데이터 추출
        script = item.get('consulting_content', "")

        print(f"테스트 실행 중: {item.get('source_id', 'Unknown ID')}")
        res, metrics = await get_personality(script)
            
        # 결과 저장
        results_list.append({
            "id": item.get('source_id'),
            "res": res,
            "metrics": metrics,
        })

테스트 실행 중: 20593
테스트 실행 중: 20594


In [7]:
import os
import pandas as pd

df_new = pd.json_normalize(results_list)
df_new['model'] = "EXAONE-3.5-2.4B-Instruct" 

output_file = "evaluation_results_personality.csv"

if not os.path.exists(output_file):
    df_new.to_csv(output_file, index=False, mode='w', encoding='utf-8-sig')
else:
    df_new.to_csv(output_file, index=False, mode='a', encoding='utf-8-sig', header=False)

print(f"데이터가 추가되었습니다: {output_file}")

데이터가 추가되었습니다: evaluation_results_personality.csv
